# 04 Flow Sorted Portfolios


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [ ]:
from src.signals.flow_sorted_factor import build_flow_sorted_portfolios

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

result = build_flow_sorted_portfolios(
    returns=returns,
    flow=flow,
    rebalance_dates=rebalance_dates,
    q=cfg["flow_sorted_factor"]["q"],
    weighting=cfg["flow_sorted_factor"]["weighting"],
    lag=cfg["flow_sorted_factor"]["lag_days"],
)

bucket_returns = result["bucket_returns"]
ls_return = result["ls_return"]
turnover = result["turnover"]

summary = pd.DataFrame({
    "mean": bucket_returns.mean(),
    "vol": bucket_returns.std(ddof=0),
})
summary["sharpe"] = summary["mean"] / summary["vol"].replace(0.0, np.nan) * np.sqrt(252.0)
summary.loc["LS", "mean"] = ls_return.mean()
summary.loc["LS", "vol"] = ls_return.std(ddof=0)
summary.loc["LS", "sharpe"] = ls_return.mean() / (ls_return.std(ddof=0) if ls_return.std(ddof=0) > 0 else np.nan) * np.sqrt(252.0)
summary.loc["LS", "turnover_mean"] = turnover.mean()

assert bucket_returns.shape[0] == len(dates)
assert ls_return.shape[0] == len(dates)

save_csv(summary, PROJECT_ROOT / "outputs/diagnostics/04_flow_sorted_summary.csv")
save_parquet(bucket_returns, PROJECT_ROOT / "outputs/signals/04_bucket_returns.parquet")
save_parquet(ls_return, PROJECT_ROOT / "outputs/signals/04_ls_return.parquet")

fig, ax = plt.subplots(figsize=(10, 4))
(1.0 + ls_return.fillna(0.0)).cumprod().plot(ax=ax)
ax.set_title("04 Long-Short Equity Curve")
ax.set_ylabel("Equity")
save_figure(fig, PROJECT_ROOT / "outputs/figures/04_flow_sorted_ls.png")
plt.close(fig)

print(summary)


In [ ]:
required = [
    "outputs/diagnostics/04_flow_sorted_summary.csv",
    "outputs/signals/04_bucket_returns.parquet",
    "outputs/signals/04_ls_return.parquet",
    "outputs/figures/04_flow_sorted_ls.png",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=04_flow_sorted_portfolios.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)
